In [1]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
# import openai
from dotenv import load_dotenv
import os
import shutil

In [2]:
MD_PAGES_PATH = "./markdown_pages"
CHROMA_PATH = "./chroma"

In [3]:
def generate_data_store():
    documents = load_md_pages()
    chunks = split_text(documents)
    save_to_chroma(chunks)

def load_md_pages():
    loader = DirectoryLoader(MD_PAGES_PATH, glob="*.md")
    return loader.load()

def split_text(documents: list):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,
        chunk_overlap=200,
        length_function=len,
        add_start_index=True
    )
    return text_splitter.split_documents(documents)

def save_to_chroma(chunks: list):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    db = Chroma.from_documents(chunks, OllamaEmbeddings(model="mxbai-embed-large"), persist_directory=CHROMA_PATH, collection_name ='test')
    db.persist()
    print("Saved to Chroma")

In [4]:
# generate_data_store()

# redo but with ObjectBox instead of Chroma

In [5]:
from langchain_objectbox.vectorstores import ObjectBox

files = load_md_pages()
docs = split_text(files)  # Ensure texts are within the model's context length
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
objectbox = ObjectBox.from_documents(docs, embeddings, embedding_dimensions=1024)